In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row

In [31]:
from szp_funcs import get_fio, create_full_res, print_df, load_groups, gku_inn, path_to_docs, to_double, path_to_data, szp, sum_fot_jobs, sum_fot

In [3]:
def create_res(months):
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = load_data(months[i])
        else:
            res = pd.merge(res, load_data(months[i]), how='outer', on=['inn', 'snils'])
    return res

In [4]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[(df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index().groupby('snils').max().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on=['inn', 'snils'])
    sums = sums[(sums.stv >= 1.0) & (sums.status == 'Работа') & (sums.day == 1) & (sums.status_pref == 'Работа')]
    sums = sums[['inn', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_y': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [98]:
inn_of_int = 7730026170
months = ['jan', 'feb', 'mar']

7730026170 - school 67

7733857776 - school 2097

In [36]:
res = create_res(months)

In [89]:
res[res.inn == inn_of_int].sort_values(['szp'], ascending=True).head(19)

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp
58767,7730026170,075-652-204 69,34355.43,1.0,Учитель,NaN,NaN,NaN,NaN,NaN,NaN,34355.430000
87276,7730026170,140-946-421 55,53155.09,1.0,Учитель,NaN,NaN,NaN,NaN,NaN,NaN,53155.090000
62714,7730026170,092-882-121 89,54500.00,1.0,Младший воспитатель,65690.00,1.0,Младший воспитатель,54500.00,1.0,Младший воспитатель,58230.000000
151614,7730026170,201-286-539 30,NaN,NaN,NaN,NaN,NaN,NaN,60194.00,1.0,Заведующий библиотекой,60194.000000
32147,7730026170,025-885-185 82,57696.00,1.0,Библиотекарь,65690.00,1.0,Библиотекарь,62896.00,1.0,Библиотекарь,62094.000000
123674,7730026170,134-932-958 82,NaN,NaN,NaN,65690.00,1.0,Педагог-психолог,62335.00,1.0,Педагог-психолог,64012.500000
10785,7730026170,011-243-715 87,62500.00,1.0,Помощник воспитателя,65690.00,1.0,Помощник воспитателя,64500.00,1.0,Помощник воспитателя,64230.000000
2181,7730026170,001-542-239 86,65000.00,1.0,Воспитатель,NaN,NaN,NaN,65000.00,1.0,Воспитатель,65000.000000
98707,7730026170,161-827-442 71,64923.86,1.0,Учитель,65689.86,1.0,Педагог дополнительного образования,NaN,NaN,NaN,65306.860000
106085,7730026170,182-114-587 51,65500.00,1.0,Администратор,65690.00,1.0,Администратор,65500.00,1.0,Администратор,65563.333333


In [41]:
res[res.snils == '075-652-204 69']

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp
58767,7730026170,075-652-204 69,34355.43,1.0,Учитель,NaN,NaN,NaN,NaN,NaN,NaN,34355.43


In [42]:
res[res.snils == '140-946-421 55']

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp
87276,7730026170,140-946-421 55,53155.09,1.0,Учитель,NaN,NaN,NaN,NaN,NaN,NaN,53155.09


In [40]:
res['szp'] = res.apply(lambda row: szp(row, months), axis=1)

In [43]:
cut_off = res[['inn', 'snils', 'szp']].groupby('inn').count().rename(columns={'snils': 'cnt'})
cut_off.cnt = np.ceil(cut_off.cnt / 10)
cut_off = cut_off.to_dict()['cnt']
cut_off

{5003021368: 91.0,
 5003021495: 98.0,
 5003021640: 69.0,
 5003096290: 52.0,
 5030032182: 26.0,
 5030032376: 52.0,
 5051005503: 33.0,
 5051005623: 20.0,
 5051005670: 88.0,
 5074019220: 30.0,
 5074019252: 40.0,
 5074045703: 54.0,
 7701032028: 4.0,
 7701050524: 15.0,
 7701113894: 5.0,
 7701293580: 10.0,
 7701375995: 6.0,
 7701377657: 21.0,
 7701384943: 5.0,
 7701406844: 7.0,
 7701905508: 27.0,
 7702058364: 16.0,
 7702061938: 4.0,
 7702764296: 16.0,
 7702797527: 27.0,
 7702833359: 29.0,
 7703021039: 7.0,
 7703112310: 9.0,
 7703261626: 15.0,
 7703363890: 23.0,
 7703379868: 3.0,
 7703611221: 18.0,
 7703738796: 21.0,
 7703742922: 37.0,
 7703776689: 10.0,
 7703808740: 12.0,
 7704040242: 13.0,
 7704118121: 6.0,
 7704118139: 12.0,
 7704178709: 18.0,
 7704191153: 9.0,
 7704222919: 17.0,
 7704222926: 41.0,
 7704231631: 32.0,
 7704234760: 23.0,
 7704271031: 1.0,
 7704853583: 19.0,
 7705020295: 16.0,
 7705041520: 4.0,
 7705052410: 7.0,
 7705399348: 33.0,
 7705480398: 29.0,
 7705513678: 28.0,
 770551

In [93]:
res[res.inn == inn_of_int][['inn', 'snils', 'szp']].groupby('inn').count().rename(columns={'snils': 'cnt'})

,cnt,szp
inn,,
7733857776,393,393


In [99]:
cut_off[inn_of_int]

18.0

In [46]:
ppl = res[['inn', 'szp', 'snils']].dropna().sort_values('szp').groupby('inn')
keys = ppl.groups.keys()
val_1_7 = pd.DataFrame({'inn': [], 'val': []})

In [80]:
ppl.get_group(inn_of_int)

,inn,szp,snils
149858,7733857776,33129.600000,171-321-362 26
135026,7733857776,34739.200000,023-743-832 42
56995,7733857776,40824.000000,072-579-280 92
110088,7733857776,43860.750000,202-993-114 51
137301,7733857776,45824.000000,038-745-648 95
...,...,...,...
55764,7733857776,367532.000000,070-296-987 86
89240,7733857776,369544.666667,143-482-585 69
50746,7733857776,369544.666667,060-823-149 37
8033,7733857776,370705.100000,006-504-008 96


In [61]:
fio = get_fio()

In [62]:
ungrouped_ppl = pd.DataFrame({'inn': [], 'snils': [], 'szp': []})

In [63]:
for key in keys:
    ungrouped_ppl = pd.concat([ungrouped_ppl, ppl.get_group(key)[['inn', 'snils', 'szp']]])

In [67]:
ungrouped_ppl

,inn,snils,szp
118522,5.003021e+09,065-107-311 28,21566.76
126472,5.003021e+09,157-113-833 53,23419.93
121306,5.003021e+09,115-609-273 46,25150.86
99628,5.003021e+09,164-008-934 53,45556.36
111766,5.003021e+09,004-721-809 16,51513.16
...,...,...,...
99253,9.718071e+09,163-098-582 87,484642.00
90056,9.718071e+09,145-075-584 65,484642.00
138207,9.718071e+09,047-832-306 66,494642.00
150158,9.718071e+09,174-487-457 12,527029.00


In [69]:
ungrouped_ppl = pd.merge(ungrouped_ppl, fio, how='left', on='snils').drop_duplicates(['inn', 'snils'])

In [97]:
ungrouped_ppl[ungrouped_ppl.inn == '140-946-421 55']

,inn,snils,szp,fio,cut


In [73]:
ungrouped_ppl['cut'] = ungrouped_ppl.apply(lambda row: cut_off[row['inn']], axis=1)

In [75]:
print_df(ungrouped_ppl[['inn', 'cut', 'snils', 'fio', 'szp']], 'redezil ppl')

In [48]:
for key in keys:
    top = ppl.get_group(key).tail(int(cut_off[key]))['szp'].mean()
    bot = ppl.get_group(key).head(int(cut_off[key]))['szp'].mean()
    t_df = pd.DataFrame({'inn': [key], 'val': [top / bot]})
    val_1_7 = pd.concat([val_1_7, t_df])

In [100]:
ppl.get_group(inn_of_int).tail(int(cut_off[inn_of_int]))['szp'].mean()

238627.38490740737

In [95]:
ppl.get_group(inn_of_int).head(int(cut_off[inn_of_int]))['szp'].mean()

53616.19354166667

In [20]:
print_df(ppl.get_group(inn_of_int), 'redezil 67')

In [96]:
val_1_7[val_1_7.inn == inn_of_int]

,inn,val
0,7.733858e+09,3.7558


In [25]:
print_df(val_1_7, 'redezil all')

In [6]:
fot_res = create_full_res(months)

In [7]:
job_of_int = ['Педагог дополнительного образования']

In [9]:
fot_res['fot'] = fot_res.apply(lambda row: sum_fot_jobs(row, months, job_of_int), axis=1)

In [15]:
print_df(fot_res[fot_res.job_mar == job_of_int[0]][['snils', 'fot']], 'snils_for_fot')

In [ ]:
fot_res[]

In [27]:
ped, isp, aup = load_groups('input')

In [28]:
fot_res['fot_isp'] = fot_res.apply(lambda row: sum_fot_jobs(row, months, isp), axis=1)

In [33]:
fot_res['fot_all'] = fot_res.apply(lambda row: sum_fot(row, months), axis=1)

In [29]:
fot_res[fot_res.fot_isp > 0][['fot_isp']].describe()

,fot_isp
count,1.333690e+05
mean,2.771900e+05
std,1.146443e+05
min,8.060000e+00
25%,1.909870e+05
50%,2.666965e+05
75%,3.548044e+05
max,1.234783e+06


In [30]:
float(fot_res[fot_res.fot_isp > 0][['fot_isp']].sum())

36968550357.84

In [34]:
fot_res[fot_res.fot_all > 0][['fot_all']].describe()

,fot_all
count,1.716610e+05
mean,2.874996e+05
std,1.489354e+05
min,8.060000e+00
25%,1.906062e+05
50%,2.670117e+05
75%,3.589236e+05
max,3.974492e+06


In [35]:
float(fot_res[fot_res.fot_all > 0][['fot_all']].sum())

49352476051.829994

In [26]:
fot_res[fot_res.fot > 0][['fot']].describe()

,fot
count,5872.000000
mean,206610.418745
std,117013.623541
min,8.060000
25%,124883.660000
50%,193334.755000
75%,272884.982500
max,929016.000000
